### Workflow for getting composition for each roi annotation for each DLmask (DLmask generated from entire mask of ndpi):

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# quick pandas work to see if ndpi's in image list have the DL segmented masks!
# read xml files for roi annotations
roi_src = r'\\shelter\Kyu\skin_aging\clue_cohort\annotations\roi'
# read DL mask files for DL segmented masks
dl_src = r'\\shelter\Kyu\skin_aging\clue_cohort\DLmask1um'
# read image list and fill in DL_segmented column (0=DLmask missing, 1=DLmask exists)
im_list = r'\\shelter\Kyu\skin_aging\clue_cohort\CLUE_image_list_230207.xlsx'

In [3]:
roi_list = [x for x in os.listdir(roi_src) if x.endswith('.xml')] #301 rows
dl_list = [x for x in os.listdir(dl_src) if x.endswith('.tif')] #301 rows
roi_list = [x.replace(".xml","") for x in roi_list]
dl_list = [x.replace(".tif","") for x in roi_list]

In [4]:
image_list_csv_raw = pd.read_excel(r'\\shelter\Kyu\skin_aging\clue_cohort\CLUE_image_list_230207.xlsx')
image_list_csv = image_list_csv_raw[image_list_csv_raw["student score"]>1]
image_list_raw = image_list_csv_raw.filename
image_list_csv_raw = image_list_csv_raw["filename_raw"] = image_list_raw
image_list_csv_raw = image_list_csv_raw.drop(columns=["DL_segmented"])
image_list_csv #215 rows, filtered by student score > 1 (2,3,4,5 used, 0,1 unused)

,index,gender,age,pid,path,filename,year,sample id,label name,sub label,...,Inbal comment,ROI ANNOTATED (0 :not annotated 1.annotated),roi reviewed,Nroi,"hypodermis (1=yes, 0=no)",has hair,DV200,use_visium,DL_annotated,DL_segmented
13,14,M,63.301370,8,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-07 17.10.29.ndpi,2,17359,02-17359,2,...,NaN,1,1.0,NaN,1,NaN,NaN,NaN,NaN,NaN
14,15,M,63.301370,8,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-07 17.15.33.ndpi,2,17359,02-17359,1,...,left two are good within purple,1,1.0,NaN,0,NaN,NaN,NaN,NaN,NaN
52,53,M,64.147945,27,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-07 16.05.45.ndpi,3,4359,03-4359,3,...,could use the left aspect of the left piece (b...,1,1.0,NaN,1,NaN,NaN,NaN,NaN,NaN
53,54,M,64.147945,27,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-09 20.12.48.ndpi,3,4359,03-4359,1,...,NaN,1,1.0,NaN,1,NaN,NaN,NaN,NaN,NaN
54,55,M,64.147945,27,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-09 20.18.08.ndpi,3,4359,03-4359,2,...,NaN,1,1.0,NaN,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155,1156,F,79.035616,476,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-07 14.56.13.ndpi,10,5202,10-5202,A2,...,NaN,1,1.0,NaN,1,NaN,NaN,NaN,NaN,NaN
1157,1158,F,79.035616,476,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-08 16.34.23.ndpi,10,5202,10-5202,A3,...,NaN,1,1.0,NaN,1,NaN,NaN,NaN,NaN,NaN
1171,1172,M,68.202740,483,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-08 16.05.29.ndpi,10,6524,10-6524,A,...,NaN,1,1.0,NaN,1,NaN,NaN,NaN,NaN,NaN
1172,1173,M,92.435616,484,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-07 17.24.15.ndpi,10,6634,10-6634,7,...,NaN,1,1.0,NaN,0,NaN,NaN,NaN,NaN,NaN


In [ ]:
image_list_csv_raw

In [ ]:
image_list = image_list_csv.filename
image_list = [x.replace(".ndpi","") for x in image_list]
#note: all im_list images have roi annotated

In [ ]:
common = set(roi_list).intersection(dl_list)
len(common)

In [ ]:
no_segmask = set(common).difference(set(image_list)) # this is labeled 0, no segmented_mask for these 86 images
len(no_segmask) #86

In [ ]:
yes_segmask = set(common).difference(set(no_segmask))
len(yes_segmask) #215, 215+86 = 301 (total len)

In [ ]:
no_segmask

In [ ]:
index_no_segmask =

In [ ]:
# now write 0 or 1 accordingly in DL_segmented:
for range(0,len())
    image_list_csv_raw.at[13, 'DL_segmented'] = 1

In [14]:

# read a first DLmask and its corresponding roi xml file
roi_png_dst = os.path.join(roi_src,'roi_png_1x')

for DLmask in DLmasks:
    DLarr = Image.open(DLmask)
    roi_fn = DLmask.replace("png","xml")
    roi_xml = xml2df(os.path.join(roi_src,roi_fn))
    roi_arr = df2png(roi_xml) #for example: "\\shelter\Kyu\skin_aging\clue_cohort\annotations\roi\labeledmask_20rsf"
    # roi_png is an image with grayscale labeled mask, save this.
    roi_png = Image.fromarray(roi_arr)
    roi_png.save(os.path.join(roi_png_dst,roi_fn.replace("xml","png")))
    for sectionID in range(1,np.max(roi_png)):
        DL_roi = DLarr[roi_png==sectionID]
        composition = dl2comp(DL_roi)
        comps.append(compositions)


# output composition data by roi
xl_dst = r"\\shelter\Kyu\skin_aging\clue_cohort\CLUE_morphometrics_230207.xlsx"

NameError: name 'DLmasks' is not defined